# 04 Related Artists Creation

In [2]:
import pandas as pd
import sys
import numpy as np
import os
import requests
import datetime
import base64
import json
import re
from collections import Counter
import psycopg2
import tqdm
from bs4 import BeautifulSoup
from importlib import reload
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# We create a utils file with a class to store the basic commands and client ID for the Spotify API
import aux_utils
#reload(aux_utils)

In [81]:
# We have created a list of functions in db_utils.py
import db_utils
reload(db_utils)

<module 'db_utils' from '/Users/david/Google Drive/16. Master BigData/5 - Modulos/Modulo 10 - TFM/2. TFM/Codigos/Spotify/code/db_utils.py'>

### Spotify API Connection

In [5]:
# Instantiate the class
spotify_utils = aux_utils.spotify_utils
sp_ut = spotify_utils()

# Use the method connect to connect with the credentials
sp = sp_ut.connect()

### Database connection 

In [42]:
conn = psycopg2.connect("dbname=spotify")
cursor= conn.cursor()

In [10]:
# Extrack all the artists
df_artist = db_utils.select_table(cursor, table="master_artist", sel_cols=["artist_id", "artist_name", "is_main"])

In [26]:
set_artist = set(list(df_artist.artist_id))

# 1. Spotify Related Artists

In [35]:
values = []

In [36]:
for i, row in tqdm.tqdm_notebook(df_artist.iterrows()):
    
    # Artist ID
    artist_id = row.artist_id
    
    # Query to SPOTIFY API
    rel_artist_json = sp.artist_related_artists(artist_id)
    
    # Parse the json response
    list_artists = rel_artist_json["artists"]
    
    for each_artist in list_artists:
        
        #id of the related artist
        rel_art_id = each_artist["id"] 
        
        # Check that the artist_ID is in the master_artist
        if rel_art_id not in set_artist:
            continue
        
        # Popularity of rel_art
        popularity = each_artist["popularity"]
        
        # Genre loop
        genres = each_artist["genres"]
        
        for genre in genres:
            
            if len(genre) and type(genre)==str :
                
                # Clean the genre text
                genre = genre.replace("'", " ")
                genre = genre.lower()
                
                # Inser into values
                values.append((artist_id, rel_art_id, genre, popularity))

/Users/david/.local/share/virtualenvs/Spotify-xJiBt4R8/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [37]:
cpy = np.array(values, copy = True)

In [39]:
query = cursor.executemany("""insert into REL_ARTIST_ARTIST VALUES (%s, %s, %s, %s) ON CONFLICT DO NOTHING; """ , values)
conn.commit()

### Get Popular genres

In [ ]:
# SQL
#select * from (select genre, count(*) as nums from rel_artist_artist group by genre) x1 order by nums desc;